# Notebook for Reanalysing Benchmarks 

Having a look at the benchmarking simulations I ran at the beginning of the PhD and seeing if I can analyse them in a similar way to now. 

In [2]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu

In [3]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [13]:
spice_dir = pth.Path('/home/jleland/data/external/spice')
os.chdir(spice_dir)

In [5]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [29]:
skippable_scans = set(['cumulus/lowdens_anglescan']) 
single_sims = set()

In [37]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/new_sheath_exp/*'
scans_searchstr = 'cumulus/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
# angles_search_str = '/*[!.{yml, inp}]'
angles_search_str = '/*'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
        print(all_run_dirs[scan])



scans = list(scans)
scans.sort()

['cumulus/benchmarking_sam/disttest_fullnogap', 'cumulus/benchmarking_sam/disttest_fullnogapcart', 'cumulus/benchmarking_sam/disttestsmall_fullnogapcart', 'cumulus/benchmarking_sam/gapless_fullgrid', 'cumulus/benchmarking_sam/gapless_halfgrid1', 'cumulus/benchmarking_sam/gapless_halfgrid2', 'cumulus/benchmarking_sam/nogaphalfgrid_tlong1', 'cumulus/benchmarking_sam/prebprobe2_fullgap', 'cumulus/benchmarking_sam/prebprobe2_fullnogap', 'cumulus/benchmarking_sam/prebprobe_fullgap', 'cumulus/benchmarking_sam/prebprobe_fullnogap']


In [39]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: cumulus/benchmarking_sam
	[0,0]: benchmarking_sam/disttest_fullnogap
	[0,1]: benchmarking_sam/disttest_fullnogapcart
	[0,2]: benchmarking_sam/disttestsmall_fullnogapcart
	[0,3]: benchmarking_sam/gapless_fullgrid
	[0,4]: benchmarking_sam/gapless_halfgrid1
	[0,5]: benchmarking_sam/gapless_halfgrid2
	[0,6]: benchmarking_sam/nogaphalfgrid_tlong1
	[0,7]: benchmarking_sam/prebprobe2_fullgap
	[0,8]: benchmarking_sam/prebprobe2_fullnogap
	[0,9]: benchmarking_sam/prebprobe_fullgap
	[0,10]: benchmarking_sam/prebprobe_fullnogap


In [41]:
spl_path = spice_dir / all_run_dirs[scans[0]][7]
print(spl_path)

/home/jleland/data/external/spice/cumulus/benchmarking_sam/prebprobe2_fullgap


In [59]:
gap_spl_path = spice_dir / all_run_dirs[scans[0]][9]
nogap_spl_path = spice_dir / all_run_dirs[scans[0]][10]

In [60]:
gap_spl = spl.Splopter(gap_spl_path, reduce=desired_variables, #ignore_tzero_fl=True,
                       store_dataframe_fl=True, check_voltage_error_fl=False)
gap_spl.parse_input()
gap_spl.prepare(find_se_temp_fl=False,)

In [61]:
fig, ax = plt.subplots()

ax.plot(gap_spl.tdata.t_dict['ProbePot'])

In [62]:
gap_spl.iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [64]:
new_iv_data = gap_spl.iv_data.get_below_floating()
se_fitter = fts.IonCurrentSEFitter()

v_f = gap_spl.iv_data.get_vf()

fig, ax = plt.subplots()
ax.plot(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])

fit_data = se_fitter.fit(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])
fit_data.plot(ax=ax)

## Now for the non-gapped case

In [65]:
nogap_spl = spl.Splopter(nogap_spl_path, reduce=desired_variables, #ignore_tzero_fl=True,
                         store_dataframe_fl=True, check_voltage_error_fl=False)
nogap_spl.parse_input()
nogap_spl.prepare(find_se_temp_fl=False,)

In [66]:
nogap_spl.iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [67]:
fig, ax = plt.subplots()

gap_spl.iv_data.plot(ax=ax)
nogap_spl.iv_data.plot(ax=ax)

<ErrorbarContainer object of 3 artists>

In [70]:
new_iv_data = nogap_spl.iv_data.get_below_floating()
se_fitter = fts.IonCurrentSEFitter()

v_f = nogap_spl.iv_data.get_vf()

fig, ax = plt.subplots()
ax.plot(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])

fit_data = se_fitter.fit(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])
fit_data.plot(ax=ax)